# 如何使用hyperopt对xgboost进行自动调参

本教程重点在于传授如何使用Hyperopt对xgboost进行自动调参。但是这份代码也是我一直使用的代码模板之一，所以在其他数据集上套用该模板也是十分容易的。

同时因为xgboost，lightgbm，catboost。三个类库调用方法都比较一致，所以在本部分结束之后，我们有理由相信，你将会学会在这三个类库上使用hyperopt。除此之外要额外说明的是，本文并不涉及交叉验证的问题，交叉验证请查看其他教程。

## 什么是Hyperopt？

Hyperopt：是python中的一个用于"分布式异步算法组态/超参数优化"的类库。使用它我们可以拜托繁杂的超参数优化过程，自动获取最佳的超参数。广泛意义上，可以将带有超参数的模型看作是一个必然的非凸函数，因此hyperopt几乎可以稳定的获取比手工更加合理的调参结果。尤其对于调参比较复杂的模型而言，其更是能以远快于人工调参的速度同样获得远远超过人工调参的最终性能。

## 文档地址？

目前中文文档的地址由本人FontTian在2017年翻译，但是hyperopt文档本身确实写的不怎么样。所以才有了这份教程。源代码请前往[Github教程地址下载](https://github.com/FontTian/hyperopt-doc-zh/tree/master/tutorials/zh)下载。

 - [中文文档地址](https://github.com/FontTian/hyperopt-doc-zh)
 - [FontTian的博客](https://blog.csdn.net/fontthrone)
 - [Hyperopt官方文档地址](http://hyperopt.github.io/)


# 教程

## 获取数据
这里我们使用UCI的红酒质量数据集，除此之外我还额外增加了两个特征。

In [1]:
from hyperopt import fmin, tpe, hp, partial
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, zero_one_loss
import xgboost as xgb
import pandas as pd

def GetNewDataByPandas():
    wine = pd.read_csv("../data/wine.csv")
    wine['alcohol**2'] = pow(wine["alcohol"], 2)
    wine['volatileAcidity*alcohol'] = wine["alcohol"] * wine['volatile acidity']
    y = np.array(wine.quality)
    X = np.array(wine.drop("quality", axis=1))

    columns = np.array(wine.columns)

    return X, y, columns

## 分割数据并转换
首先将数据分割为三份，一部分用于预测，训练数据则同样分成额外的两部分用于evallist参数。

同时为了加快速度和减少内存，我们将数据转换为`xgboost`自带的读取格式。

In [4]:
# Read wine quality data from file
X, y, wineNames = GetNewDataByPandas()

# split data to [[0.8,0.2],01]
x_train_all, x_predict, y_train_all, y_predict = train_test_split(X, y, test_size=0.10, random_state=100)

x_train, x_test, y_train, y_test = train_test_split(x_train_all, y_train_all, test_size=0.2, random_state=100)

dtrain = xgb.DMatrix(data=x_train,label=y_train,missing=-999.0)
dtest = xgb.DMatrix(data=x_test,label=y_test,missing=-999.0)

evallist = [(dtest, 'eval'), (dtrain, 'train')]

## 定义参数空间

使用hyperopt自带的函数定义参数空间，但是因为其`randint()`方法产生的数组范围是从0开始的，所以我额外定义了一个数据转换方法，对原始参数空间进行一次转换。

关于hyperopt中定义参数区间需要使用的函数请参考：
 - 中文地址，请点击[这里](https://github.com/FontTian/hyperopt-doc-zh/wiki/FMin)
 - 英文地址，请点击[这里](https://github.com/FontTian/hyperopt-doc-zh/blob/master/hyperopt/doc/en/FMin.md)

In [6]:
# 自定义hyperopt的参数空间
space = {"max_depth": hp.randint("max_depth", 15),
         "n_estimators": hp.randint("n_estimators", 300),
         'learning_rate': hp.uniform('learning_rate', 1e-3, 5e-1),
         "subsample": hp.randint("subsample", 5),
         "min_child_weight": hp.randint("min_child_weight", 6),
         }

def argsDict_tranform(argsDict, isPrint=False):
    argsDict["max_depth"] = argsDict["max_depth"] + 5
    argsDict['n_estimators'] = argsDict['n_estimators'] + 150
    argsDict["learning_rate"] = argsDict["learning_rate"] * 0.02 + 0.05
    argsDict["subsample"] = argsDict["subsample"] * 0.1 + 0.5
    argsDict["min_child_weight"] = argsDict["min_child_weight"] + 1
    if isPrint:
        print(argsDict)
    else:
        pass

    return argsDict

## 创建模型工厂与分数获取器

xgboost模型工厂用于生产我们需要的model，而分数获取器则是为了解耦。这样在实际的测试工作中更加套用代码和修改。

In [8]:
def xgboost_factory(argsDict):
    argsDict = argsDict_tranform(argsDict)
    
    params = {'nthread': -1,  # 进程数
              'max_depth': argsDict['max_depth'],  # 最大深度
              'n_estimators': argsDict['n_estimators'],  # 树的数量
              'eta': argsDict['learning_rate'],  # 学习率
              'subsample': argsDict['subsample'],  # 采样数
              'min_child_weight': argsDict['min_child_weight'],  # 终点节点最小样本占比的和
              'objective': 'reg:linear',
              'silent': 0,  # 是否显示
              'gamma': 0,  # 是否后剪枝
              'colsample_bytree': 0.7,  # 样本列采样
              'alpha': 0,  # L1 正则化
              'lambda': 0,  # L2 正则化
              'scale_pos_weight': 0,  # 取值>0时,在数据不平衡时有助于收敛
              'seed': 100,  # 随机种子
              'missing': -999,  # 填充缺失值
              }
    params['eval_metric'] = ['rmse']

    xrf = xgb.train(params, dtrain, 300, evallist,early_stopping_rounds=100)

    return get_tranformer_score(xrf)

def get_tranformer_score(tranformer):
    
    xrf = tranformer
    dpredict = xgb.DMatrix(x_predict)
    prediction = xrf.predict(dpredict, ntree_limit=xrf.best_ntree_limit)
  
    return mean_squared_error(y_predict, prediction)

### 调用Hyperopt开始调参
之后我们调用hyperopt进行自动调参即可，同时通过返回值获取最佳模型的结果。

In [9]:
# 开始使用hyperopt进行自动调参
algo = partial(tpe.suggest, n_startup_jobs=1)
best = fmin(xgboost_factory, space, algo=algo, max_evals=20, pass_expr_memo_ctrl=None)

[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 142 extra nodes, 0 pruned nodes, max_depth=10
[0]	eval-rmse:5.03273	train-rmse:4.90203
Multiple eval metrics have been passed: 'train-rmse' will be used for early stopping.

Will train until train-rmse hasn't improved in 100 rounds.
[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nodes, 0 pruned nodes, max_depth=10
[1]	eval-rmse:4.77384	train-rmse:4.64767
[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 166 extra nodes, 0 pruned nodes, max_depth=10
[2]	eval-rmse:4.53129	train-rmse:4.40785
[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 174 extra nodes, 0 pruned nodes, max_depth=10
[3]	eval-rmse:4.29894	train-rmse:4.17815
[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 162 extra nodes, 0 pruned nodes, max_depth=10
[4]	eval-rmse:4.08249	train-rmse:3.96285
[15:23:32] /workspace/sr

[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=10
[49]	eval-rmse:0.709126	train-rmse:0.492577
[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=10
[50]	eval-rmse:0.698495	train-rmse:0.478
[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=10
[51]	eval-rmse:0.689878	train-rmse:0.464254
[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 0 pruned nodes, max_depth=10
[52]	eval-rmse:0.682074	train-rmse:0.451864
[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 108 extra nodes, 0 pruned nodes, max_depth=10
[53]	eval-rmse:0.673183	train-rmse:0.43937
[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 0 pruned nodes, max_depth=10
[54]	eval-rmse:0.667423	train-rmse:0

[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 96 extra nodes, 0 pruned nodes, max_depth=10
[98]	eval-rmse:0.582969	train-rmse:0.252804
[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=10
[99]	eval-rmse:0.582516	train-rmse:0.251364
[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 76 extra nodes, 0 pruned nodes, max_depth=10
[100]	eval-rmse:0.582768	train-rmse:0.249875
[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 0 pruned nodes, max_depth=10
[101]	eval-rmse:0.581615	train-rmse:0.24736
[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=10
[102]	eval-rmse:0.582145	train-rmse:0.245484
[15:23:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 0 pruned nodes, max_depth=10
[103]	eval-rmse:0.581879	train-r

[146]	eval-rmse:0.579473	train-rmse:0.182676
[15:23:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=10
[147]	eval-rmse:0.579539	train-rmse:0.18119
[15:23:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 78 extra nodes, 0 pruned nodes, max_depth=10
[148]	eval-rmse:0.579887	train-rmse:0.180277
[15:23:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 102 extra nodes, 0 pruned nodes, max_depth=10
[149]	eval-rmse:0.580158	train-rmse:0.179141
[15:23:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 128 extra nodes, 0 pruned nodes, max_depth=10
[150]	eval-rmse:0.57985	train-rmse:0.177764
[15:23:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=10
[151]	eval-rmse:0.57985	train-rmse:0.177329
[15:23:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 0 pruned nodes

[15:23:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 0 pruned nodes, max_depth=10
[196]	eval-rmse:0.581901	train-rmse:0.134833
[15:23:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 72 extra nodes, 0 pruned nodes, max_depth=10
[197]	eval-rmse:0.581569	train-rmse:0.13429
[15:23:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=10
[198]	eval-rmse:0.581259	train-rmse:0.133862
[15:23:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=10
[199]	eval-rmse:0.581251	train-rmse:0.132867
[15:23:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=10
[200]	eval-rmse:0.581742	train-rmse:0.131927
[15:23:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 140 extra nodes, 0 pruned nodes, max_depth=10
[201]	eval-rmse:0.581435	trai

...

[15:24:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 562 extra nodes, 0 pruned nodes, max_depth=15
[75]	eval-rmse:0.580878	train-rmse:0.09489
[15:24:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 586 extra nodes, 0 pruned nodes, max_depth=15
[76]	eval-rmse:0.580023	train-rmse:0.090508
[15:24:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 518 extra nodes, 0 pruned nodes, max_depth=15
[77]	eval-rmse:0.579198	train-rmse:0.086473
[15:24:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 680 extra nodes, 0 pruned nodes, max_depth=15
[78]	eval-rmse:0.578395	train-rmse:0.082651
[15:24:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 382 extra nodes, 0 pruned nodes, max_depth=15
[79]	eval-rmse:0.577752	train-rmse:0.079272
[15:24:02] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 508 extra nodes, 0 pruned nodes, max_depth=15
[80]	eval-rmse:0.577199	train-r

[123]	eval-rmse:0.569438	train-rmse:0.017786
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 296 extra nodes, 0 pruned nodes, max_depth=15
[124]	eval-rmse:0.569508	train-rmse:0.017393
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 372 extra nodes, 0 pruned nodes, max_depth=15
[125]	eval-rmse:0.569472	train-rmse:0.016966
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 560 extra nodes, 0 pruned nodes, max_depth=15
[126]	eval-rmse:0.569459	train-rmse:0.016454
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 546 extra nodes, 0 pruned nodes, max_depth=15
[127]	eval-rmse:0.56941	train-rmse:0.01605
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 338 extra nodes, 0 pruned nodes, max_depth=15
[128]	eval-rmse:0.56937	train-rmse:0.015665
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 360 extra nodes, 0 pruned nod

[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 366 extra nodes, 0 pruned nodes, max_depth=15
[173]	eval-rmse:0.569842	train-rmse:0.006134
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 220 extra nodes, 0 pruned nodes, max_depth=15
[174]	eval-rmse:0.569869	train-rmse:0.006038
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 392 extra nodes, 0 pruned nodes, max_depth=15
[175]	eval-rmse:0.569901	train-rmse:0.005909
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 420 extra nodes, 0 pruned nodes, max_depth=15
[176]	eval-rmse:0.569916	train-rmse:0.005784
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 582 extra nodes, 0 pruned nodes, max_depth=15
[177]	eval-rmse:0.569898	train-rmse:0.005633
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 552 extra nodes, 0 pruned nodes, max_depth=15
[178]	eval-rmse:0.569879	

[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 382 extra nodes, 0 pruned nodes, max_depth=15
[222]	eval-rmse:0.570134	train-rmse:0.002271
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 256 extra nodes, 0 pruned nodes, max_depth=15
[223]	eval-rmse:0.570147	train-rmse:0.002233
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 210 extra nodes, 0 pruned nodes, max_depth=15
[224]	eval-rmse:0.570159	train-rmse:0.002204
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 428 extra nodes, 0 pruned nodes, max_depth=15
[225]	eval-rmse:0.570174	train-rmse:0.002156
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 180 extra nodes, 0 pruned nodes, max_depth=15
[226]	eval-rmse:0.570193	train-rmse:0.002123
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 312 extra nodes, 0 pruned nodes, max_depth=15
[227]	eval-rmse:0.570192	

[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=15
[271]	eval-rmse:0.570338	train-rmse:0.001026
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 0 pruned nodes, max_depth=15
[272]	eval-rmse:0.570345	train-rmse:0.001017
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 258 extra nodes, 0 pruned nodes, max_depth=15
[273]	eval-rmse:0.570347	train-rmse:0.000993
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 0 pruned nodes, max_depth=15
[274]	eval-rmse:0.570353	train-rmse:0.000982
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 238 extra nodes, 0 pruned nodes, max_depth=15
[275]	eval-rmse:0.570355	train-rmse:0.000962
[15:24:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 228 extra nodes, 0 pruned nodes, max_depth=15
[276]	eval-rmse:0.57036	tra

## 展示结果
展示我们获取的最佳参数，以及该模型在训练集上的最终表现，如果想要使用交叉验证请参考其他教程。

In [10]:
RMSE = xgboost_factory(best)
print('best :', best)
print('best param after transform :')
argsDict_tranform(best,isPrint=True)
print('rmse of the best xgboost:', np.sqrt(RMSE))

[15:24:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 428 extra nodes, 0 pruned nodes, max_depth=14
[0]	eval-rmse:5.02286	train-rmse:4.89385
Multiple eval metrics have been passed: 'train-rmse' will be used for early stopping.

Will train until train-rmse hasn't improved in 100 rounds.
[15:24:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 680 extra nodes, 0 pruned nodes, max_depth=14
[1]	eval-rmse:4.75938	train-rmse:4.63251
[15:24:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 616 extra nodes, 0 pruned nodes, max_depth=14
[2]	eval-rmse:4.50827	train-rmse:4.38475
[15:24:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 626 extra nodes, 0 pruned nodes, max_depth=14
[3]	eval-rmse:4.27187	train-rmse:4.14922
[15:24:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 632 extra nodes, 0 pruned nodes, max_depth=14
[4]	eval-rmse:4.04979	train-rmse:3.92771
[15:24:52] /workspace/sr

[15:24:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 562 extra nodes, 0 pruned nodes, max_depth=14
[49]	eval-rmse:0.689545	train-rmse:0.3422
[15:24:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 666 extra nodes, 0 pruned nodes, max_depth=14
[50]	eval-rmse:0.679815	train-rmse:0.324627
[15:24:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 598 extra nodes, 0 pruned nodes, max_depth=14
[51]	eval-rmse:0.671419	train-rmse:0.307946
[15:24:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 636 extra nodes, 0 pruned nodes, max_depth=14
[52]	eval-rmse:0.66369	train-rmse:0.292147
[15:24:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 608 extra nodes, 0 pruned nodes, max_depth=14
[53]	eval-rmse:0.656612	train-rmse:0.277276
[15:24:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 480 extra nodes, 0 pruned nodes, max_depth=14
[54]	eval-rmse:0.650339	train-rms

[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 426 extra nodes, 0 pruned nodes, max_depth=14
[98]	eval-rmse:0.584092	train-rmse:0.039064
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 490 extra nodes, 0 pruned nodes, max_depth=14
[99]	eval-rmse:0.583889	train-rmse:0.03776
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 540 extra nodes, 0 pruned nodes, max_depth=14
[100]	eval-rmse:0.583817	train-rmse:0.036501
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 314 extra nodes, 0 pruned nodes, max_depth=14
[101]	eval-rmse:0.583798	train-rmse:0.035645
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 440 extra nodes, 0 pruned nodes, max_depth=14
[102]	eval-rmse:0.583548	train-rmse:0.034537
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 540 extra nodes, 0 pruned nodes, max_depth=14
[103]	eval-rmse:0.583439	tra

[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 464 extra nodes, 0 pruned nodes, max_depth=14
[147]	eval-rmse:0.58287	train-rmse:0.011141
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 446 extra nodes, 0 pruned nodes, max_depth=14
[148]	eval-rmse:0.582913	train-rmse:0.010857
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 612 extra nodes, 0 pruned nodes, max_depth=14
[149]	eval-rmse:0.582931	train-rmse:0.010548
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 618 extra nodes, 0 pruned nodes, max_depth=14
[150]	eval-rmse:0.583001	train-rmse:0.010267
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 296 extra nodes, 0 pruned nodes, max_depth=14
[151]	eval-rmse:0.582983	train-rmse:0.010087
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 600 extra nodes, 0 pruned nodes, max_depth=14
[152]	eval-rmse:0.582963	t

[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 296 extra nodes, 0 pruned nodes, max_depth=14
[196]	eval-rmse:0.5835	train-rmse:0.003985
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 430 extra nodes, 0 pruned nodes, max_depth=14
[197]	eval-rmse:0.583512	train-rmse:0.003898
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 346 extra nodes, 0 pruned nodes, max_depth=14
[198]	eval-rmse:0.583529	train-rmse:0.003806
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 206 extra nodes, 0 pruned nodes, max_depth=14
[199]	eval-rmse:0.583546	train-rmse:0.003752
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 316 extra nodes, 0 pruned nodes, max_depth=14
[200]	eval-rmse:0.583559	train-rmse:0.003677
[15:24:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 314 extra nodes, 0 pruned nodes, max_depth=14
[201]	eval-rmse:0.583564	tr

[244]	eval-rmse:0.583786	train-rmse:0.001599
[15:24:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 274 extra nodes, 0 pruned nodes, max_depth=14
[245]	eval-rmse:0.583799	train-rmse:0.001574
[15:24:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 272 extra nodes, 0 pruned nodes, max_depth=14
[246]	eval-rmse:0.583791	train-rmse:0.00154
[15:24:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 244 extra nodes, 0 pruned nodes, max_depth=14
[247]	eval-rmse:0.583792	train-rmse:0.001511
[15:24:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 292 extra nodes, 0 pruned nodes, max_depth=14
[248]	eval-rmse:0.583785	train-rmse:0.001479
[15:24:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=14
[249]	eval-rmse:0.583794	train-rmse:0.001464
[15:24:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 270 extra nodes, 0 pruned n

[15:24:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 176 extra nodes, 0 pruned nodes, max_depth=14
[294]	eval-rmse:0.583917	train-rmse:0.000745
[15:24:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 0 pruned nodes, max_depth=14
[295]	eval-rmse:0.583917	train-rmse:0.000732
[15:24:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=14
[296]	eval-rmse:0.583917	train-rmse:0.000725
[15:24:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 64 extra nodes, 0 pruned nodes, max_depth=12
[297]	eval-rmse:0.583918	train-rmse:0.000719
[15:24:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 186 extra nodes, 0 pruned nodes, max_depth=14
[298]	eval-rmse:0.583923	train-rmse:0.000705
[15:24:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=7
[299]	eval-rmse:0.583926	trai